In [ ]:
%cd ../..

# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
import time
from os import environ

In [ ]:
if "AP_PORT" not in environ:
    environ["AP_PORT"] = "25432" #str(input('port'))

In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F
from apperception.predicate import camera, objects, lit
database.connection

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (F.like(cam.filename, 'samples/CAM_FRONT/%')) & 
    (obj1.id != obj2.id) &
    (F.like(obj1.type, 'vehicle.car') | F.like(obj1.type, 'vehicle.truck')) & ########
    (F.like(obj2.type, 'vehicle.car') | F.like(obj2.type, 'vehicle.truck')) & ########
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) & ########
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) & #########
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) & ########
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) & #########
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) & ########
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) & ########
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 50, 135) & ########
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) & ########
    # (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100) ####
)


start = time.time()

id_time_camId_filename = world.get_id_time_camId_filename(2)

end = time.time()
print(format(end-start))

In [ ]:
list(set([x[4] for x in id_time_camId_filename if "samples" in x[4]]))

In [ ]:
start = time.time()
res = database.sql("""
SELECT table_0.itemId,table_1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as table_0
        JOIN (SELECT * FROM item_general_trajectory) as table_1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE (((Cameras.cameraId LIKE 'scene-0000') OR (Cameras.cameraId LIKE 'scene-0001') OR (Cameras.cameraId LIKE 'scene-0002') OR (Cameras.cameraId LIKE 'scene-0003') OR (Cameras.cameraId LIKE 'scene-0004')) AND (table_0.itemId<>table_1.itemId) AND (table_0.objectType LIKE 'vehicle%') AND (table_1.objectType LIKE 'vehicle%') AND angleBetween(facingRelative(Cameras.egoHeading, roadDirection(Cameras.egoTranslation, Cameras.egoHeading)), (-15), 15) AND (distance(Cameras.egoTranslation, table_0.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_0.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<(70/2.0)) AND (distance(Cameras.egoTranslation, table_1.trajCentroids, Cameras.timestamp)<50) AND (viewAngle(table_1.trajCentroids, Cameras.egoHeading, Cameras.egoTranslation, Cameras.timestamp)<(70/2.0)) AND (EXISTS(
        SELECT intersection.id
        FROM intersection
            JOIN SegmentPolygon
                ON SegmentPolygon.elementId = intersection.id
            JOIN unnest(ARRAY[valueAtTimestamp(table_0.trajCentroids,Cameras.timestamp),valueAtTimestamp(table_1.trajCentroids,Cameras.timestamp)]) point
                ON ST_Covers(SegmentPolygon.elementPolygon, point)
        GROUP BY intersection.id
        HAVING COUNT(point) = 2
    )) AND angleBetween(facingRelative(table_0.itemHeadings, Cameras.egoHeading, Cameras.timestamp), 50, 135) AND angleBetween(facingRelative(table_1.itemHeadings, Cameras.egoHeading, Cameras.timestamp), (-135), (-50)) AND (minDistance(Cameras.egoTranslation, roadSegment('intersection'))<10) AND angleBetween(facingRelative(table_0.itemHeadings, table_1.itemHeadings, Cameras.timestamp), 100, (-100)) AND (1=1))
""")
end = time.time()
res

In [ ]:
end - start

In [ ]:
import json
print(json.dumps(database._execute_query("""
EXPLAIN (ANALYZE, COSTS, VERBOSE, BUFFERS, FORMAT JSON)
SELECT t0.itemId,t1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.itemId<>t1.itemId) AND t0.objectType LIKE 'vehicle%' AND t1.objectType LIKE 'vehicle%' AND angleBetween(facingRelative(egoHeading,roadDirection),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t1.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t1.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (EXISTS(SELECT 1
            FROM intersection
            JOIN SegmentPolygon
                ON SegmentPolygon.elementId = intersection.id
            JOIN unnest(ARRAY[valueAtTimestamp(t0.trajCentroids,Cameras.timestamp),valueAtTimestamp(t1.trajCentroids,Cameras.timestamp)]) point
                ON ST_Covers(SegmentPolygon.elementPolygon, point)
        GROUP BY intersection.id
        HAVING COUNT(point) = 2
        )) AND angleBetween(facingRelative((valueAtTimestamp(t0.itemHeadings,timestamp))::real,egoHeading),50,135) AND angleBetween(facingRelative((valueAtTimestamp(t1.itemHeadings,timestamp))::real,egoHeading),-135,-50) AND (minDistance(egoTranslation,'intersection')<10) AND angleBetween(facingRelative((valueAtTimestamp(t0.itemHeadings,timestamp))::real,(valueAtTimestamp(t1.itemHeadings,timestamp))::real),100,-100))
""")))

In [ ]:
import json
print(json.dumps(database._execute_query("""
EXPLAIN (ANALYZE, COSTS, VERBOSE, BUFFERS, FORMAT JSON)
        SELECT t0.itemId,t1.itemId, cameras.timestamp, cameras.cameraId, cameras.filename
        FROM (SELECT * FROM item_general_trajectory) as t0
        JOIN (SELECT * FROM item_general_trajectory) as t1 USING (cameraId)
        JOIN Cameras USING (cameraId)
        WHERE ((t0.itemId<>t1.itemId) AND t0.objectType LIKE 'vehicle%' AND t1.objectType LIKE 'vehicle%' AND angleBetween(facingRelative(egoHeading,roadDirection),-15,15) AND (ST_Distance(egoTranslation,valueAtTimestamp(t0.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t0.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (ST_Distance(egoTranslation,valueAtTimestamp(t1.translations,timestamp))<50) AND (viewAngle(valueAtTimestamp(t1.translations,timestamp),egoHeading,egoTranslation)<35.0) AND (EXISTS(SELECT 1
            FROM SegmentPolygon
            WHERE
                SegmentPolygon.__RoadType__intersection__ AND
                ST_Covers(SegmentPolygon.elementPolygon, valueAtTimestamp(t0.translations,timestamp)) AND ST_Covers(SegmentPolygon.elementPolygon, valueAtTimestamp(t1.translations,timestamp))
        ))
        AND angleBetween(facingRelative((valueAtTimestamp(t0.itemHeadings,timestamp))::real,egoHeading),50,135)
        AND angleBetween(facingRelative((valueAtTimestamp(t1.itemHeadings,timestamp))::real,egoHeading),-135,-50)
        AND (EXISTS ( SELECT 1
        FROM SegmentPolygon 
        WHERE SegmentPolygon.__RoadType_intersection__ AND ST_Distance(Cameras.egoTranslation, elementPolygon) < 10))
        AND angleBetween(facingRelative((valueAtTimestamp(t0.itemHeadings,timestamp))::real,(valueAtTimestamp(t1.itemHeadings,timestamp))::real),100,-100))
""")))

In [ ]:
start = time.time()
res = database.sql("""
WITH car as (
SELECT *
FROM item_general_trajectory
  JOIN Cameras USING (cameraId)
WHERE
  objectType LIKE 'vehicle%'
  AND (cameraID = 'scene-0000' OR cameraID = 'scene-0001' OR cameraID = 'scene-0002' OR cameraID = 'scene-0003' OR cameraID = 'scene-0004')
    AND (
      distance(
        Cameras.egoTranslation, trajCentroids, 
        Cameras.timestamp
      )< 50
    ) 
    AND (
      viewAngle(
        trajCentroids, Cameras.egoHeading, 
        Cameras.egoTranslation, Cameras.timestamp
      )<(70 / 2.0)
    ) AND
    angleBetween(
      facingRelative(
        Cameras.egoHeading, 
        roadDirection(
          Cameras.egoTranslation, Cameras.egoHeading
        )
      ), 
      (-15), 
      15
    ) 
    AND (
      minDistance(
        Cameras.egoTranslation, 
        roadSegment('intersection')
      )< 10
    ) 
), inters AS (
    SELECT elementPolygon, id
    FROM Intersection
    JOIN SegmentPolygon ON elementId = id
), car1 AS (
    SELECT *
    FROM car
    WHERE
    angleBetween(
      facingRelative(
        itemHeadings, egoHeading, 
        timestamp
      ), 
      50, 
      135
    ) 
), car2 AS (
    SELECT *
    FROM car
    WHERE
    angleBetween(
      facingRelative(
        itemHeadings, egoHeading, 
        timestamp
      ), 
      (-135), 
      (-50)
    ) 
)

SELECT 
  table_0.itemId, 
  table_1.itemId, 
  table_0.timestamp, 
  table_0.cameraId, 
  table_0.filename 
FROM 
  car1 as table_0
  JOIN car2 as table_1 USING (cameraId, timestamp, filename) 
WHERE 
  (
    table_0.itemId <> table_1.itemId AND
    (
      EXISTS(
        SELECT 
          inters.id
        FROM 
          inters
          JOIN unnest(
            ARRAY[valueAtTimestamp(
              table_0.trajCentroids, table_0.timestamp
            ), 
            valueAtTimestamp(
              table_1.trajCentroids, table_0.timestamp
            ) ]
          ) point ON ST_Covers(
            inters.elementPolygon, point
          ) 
        GROUP BY 
          inters.id 
        HAVING 
          COUNT(point) = 2
      )
    ) 
    AND angleBetween(
      facingRelative(
        table_0.itemHeadings, table_1.itemHeadings, 
        table_0.timestamp
      ), 
      100, 
      (-100)
    )
  )
""")
end = time.time()
res

In [ ]:
from apperception.utils import overlay_trajectory
from apperception.database import database

data_dir =  "data/nuscenes/experiment_data"

overlay_trajectory(world=world, database=database, images_data_path=data_dir, is_overlay_headings=True, is_overlay_objects=True, is_overlay_road=True, is_keep_whole_video=True)

In [ ]:
from apperception.data_types import QueryType, Trajectory
import datetime
query = f"""
        SELECT asMFJSON(trajCentroids)::json->'sequences'
        FROM Item_General_Trajectory as final
        WHERE itemId = '9d03c6edb6eb4d49acccb245bdd0c652'
        """

t = database._execute_query(query)[0][0][0]
coordinates = t["coordinates"]
datetimes = t["datetimes"]
result = {}
for coord, time in zip(coordinates, datetimes):
    dt_time = datetime.datetime.strptime(time, "%Y-%m-%dT%H:%M:%S.%f+00").replace(tzinfo=datetime.timezone.utc)
    result[dt_time] = coord

print(result)


In [ ]:
# for k in id_time_camId_filename:
#     print(k[-2], str(k[-3]), k[-1])
#     print("                                           ", *k[:-3])

In [ ]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId1, itemId2, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    if filename not in resultImages:
        resultImages[filename] = []
    resultImages[filename].append((itemId1, itemId2, timestamp, camId))

In [ ]:
from show_images import show_images
%matplotlib inline

data_dir =  "data/nuscenes/experiment_data"
# data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'
# data_dir = '/Users/chanwutk/Documents/experiment_data'

In [ ]:
show_images(data_dir, resultImages, sample=10)

In [ ]:
query = f"""
CREATE OR REPLACE FUNCTION ST_XYZ (g geometry) RETURNS real[] AS $$
    BEGIN
        RETURN ARRAY[ST_X(g), ST_Y(g), ST_Z(g)];
    END;
$$ LANGUAGE plpgsql;

SELECT
    cameraId,
    ST_XYZ(egoTranslation),
    egoRotation,
    ST_XYZ(cameraTranslation),
    ST_XYZ(cameraTranslationAbs),
    cameraRotation,
    cameraIntrinsic,
    frameNum,
    fileName,
    cameraHeading,
    egoHeading
FROM Cameras
WHERE
    fileName = 'samples/CAM_FRONT/n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg'
ORDER BY cameraId ASC, frameNum ASC;
"""
# print(query)
result = database._execute_query(query)

In [ ]:
expected_imgs = [
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657119112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404.jpg',
    'n008-2018-09-18-12-07-26-0400__CAM_FRONT__1537287358412404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118612404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657128112404.jpg',
]
show_images(data_dir, expected_imgs, sample=10)

In [ ]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(data_dir, missing_images, sample=10)

In [ ]:
extra = 0
extra_images = {}
for img in resultImages.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages.keys()), "=", 100 * extra / len(resultImages.keys()), "%")
show_images(data_dir, extra_images, sample=10, seed=5)